<a href="https://colab.research.google.com/github/Ddkaba/IAD_Lab_1/blob/main/IAD_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.svm import SVC

import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
dataset = pd.read_csv("https://raw.githubusercontent.com/Ddkaba/IAD_Lab_1/main/V2.csv", index_col=0)
if 'No' in dataset.columns:
    dataset = dataset.drop(columns=['No'])

In [ ]:
# ### BEGIN YOUR CODE
print("Общая информация")
print(dataset.info())

print(f"Количество записей (объектов): {dataset.shape[0]}")
print(f"Количество признаков (фич): {dataset.shape[1]}")

print("\nНазвания столбцов:")
print(dataset.columns.tolist())

print("\nТипы данных:")
print(dataset.dtypes)

print("\nПропущенные значения:")
missing_values = dataset.isnull().sum()
print(missing_values)
print(f"Общее количество пропущенных значений: {missing_values.sum()}")

print("Целевая переменная")
target_column = 'Y house price of unit area'
if target_column in dataset.columns:
    print(f"\nЦелевая переменная: {target_column}")
    print(f"Тип данных целевой переменной: {dataset[target_column].dtype}")
    unique_values = dataset[target_column].unique()
    print(f"Уникальные значения целевой переменной (первые 20): {unique_values[:20]}")
    print(f"Всего уникальных значений: {unique_values.size}")
    if dataset[target_column].nunique() <= 20:
        print("Распределение классов:")
        print(dataset[target_column].value_counts())
        print("Процентное соотношение классов:")
        print(dataset[target_column].value_counts(normalize=True) * 100)

print("Статистика")
print(dataset.describe())

print("Анализ кат. признаков")
categorical_features = []
for col in dataset.columns:
    unique_values = dataset[col].nunique(dropna=True)
    if unique_values <= 10:
        categorical_features.append(col)
        print(f"{col}: {unique_values} уникальных значений - {dataset[col].unique()}")

print(f"\nВсего категориальных признаков: {len(categorical_features)}")

print("Корреляционная матрица (тепловая карта)")

corr_matrix = dataset.corr(numeric_only=True)
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Корреляционная матрица признаков')
plt.tight_layout()
plt.show()

print("Корреляции признаков с целевой переменной:")
if target_column in corr_matrix.columns:
    print(corr_matrix[target_column].sort_values(ascending=False))
# ### END YOUR CODE


Общая информация
<class 'pandas.core.frame.DataFrame'>
Index: 414 entries, 0 to 413
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   X1 transaction date                     414 non-null    float64
 1   X2 house age                            205 non-null    float64
 2   X3 distance to the nearest MRT station  414 non-null    float64
 3   X4 number of convenience stores         414 non-null    int64  
 4   X5 latitude                             414 non-null    float64
 5   X6 longitude                            414 non-null    float64
 6   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 25.9 KB
None
Количество записей (объектов): 414
Количество признаков (фич): 7

Названия столбцов:
['X1 transaction date', 'X2 house age', 'X3 distance to the nearest MRT station', 'X4 number of convenience stores', 'X5 l

In [ ]:
# ### BEGIN YOUR CODE
# Гистограммы по всем числовым признакам (без целевой)
# Определим список числовых колонок и исключим целевую, если она есть
feature_columns = dataset.select_dtypes(include=[np.number]).columns.tolist()
if 'Y house price of unit area' in feature_columns:
    feature_columns.remove('Y house price of unit area')

_ = dataset[feature_columns].hist(
    bins=10,
    figsize=(20, 15),
    grid=False,
    edgecolor='black'
)
plt.suptitle('Распределение числовых признаков', y=1.02)
plt.tight_layout()
plt.show()
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Геовизуализация: долгота vs широта
lon_col = 'X6 longitude'
lat_col = 'X5 latitude'

# Базовый scatter, как на слайде
ax = dataset.plot(kind='scatter', x=lon_col, y=lat_col, alpha=0.1, figsize=(8, 6))
ax.set_title('Расположение объектов (longitude vs latitude)')
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
plt.tight_layout()
plt.show()

# Раскраска по целевой (если есть)
if 'Y house price of unit area' in dataset.columns:
    plt.figure(figsize=(9, 7))
    sc = plt.scatter(
        dataset[lon_col], dataset[lat_col],
        c=dataset['Y house price of unit area'], cmap='viridis',
        s=25, alpha=0.6, edgecolors='none'
    )
    cbar = plt.colorbar(sc)
    cbar.set_label('Y house price of unit area')
    plt.title('Геоданные, окрашенные по целевой переменной')
    plt.xlabel('longitude')
    plt.ylabel('latitude')
    plt.tight_layout()
    plt.show()
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Эксперименты с комбинациями атрибутов + пересчет корреляций
TARGET = 'Y house price of unit area'

dataset_fe = dataset.copy()

# 1) Взаимодействие возраста. Старый и далёкий объект «штрафуется» сильнее, новый рядом со станцией — «премируется». Это даёт модели не одну общую «цену километра», а разную в зависимости от возраста.
age_med = dataset_fe['X2 house age'].median()
age_filled_tmp = dataset_fe['X2 house age'].fillna(age_med)
dataset_fe['age_x_mrt_distance'] = age_filled_tmp * dataset_fe['X3 distance to the nearest MRT station']

# Корреляции
corr = dataset_fe.corr(numeric_only=True)
plt.figure(figsize=(13, 10))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Корреляции после инженерии признаков')
plt.tight_layout()
plt.show()

if TARGET in corr.columns:
    print('\nТоп-15 корреляций с целевой:')
    print(corr[TARGET].sort_values(ascending=False).head(15))
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Импутация пропусков в X2 house age медианой + контроль результата

col = 'X2 house age'
med = dataset_fe[col].median()
missing_before = dataset_fe[col].isna().sum()

dataset_fe[col] = dataset_fe[col].fillna(med)
missing_after = dataset_fe[col].isna().sum()

print(f"Медиана {col}: {med:.3f}")
print(f"Пропусков до/после: {missing_before} -> {missing_after}")

# Визуальная проверка распределения после импутации
plt.figure(figsize=(6, 4))
dataset_fe[col].hist(bins=20, edgecolor='black')
plt.title(f'{col} после импутации медианой')
plt.xlabel('age')
plt.ylabel('count')
plt.tight_layout()
plt.show()

# Корреляции
corr = dataset_fe.corr(numeric_only=True)
plt.figure(figsize=(13, 10))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Корреляции после инженерии признаков')
plt.tight_layout()
plt.show()

if TARGET in corr.columns:
    print('\nТоп-15 корреляций с целевой:')
    print(corr[TARGET].sort_values(ascending=False).head(15))
# ### END YOUR CODE

In [ ]:
# ### BEGIN YOUR CODE
# SelectKBest (f_regression): оценка информативности признаков и выбор ТОП-5


TARGET = 'Y house price of unit area'

# Берём расширенный набор, если он уже посчитан; иначе — исходный
source_df = dataset_fe if 'dataset_fe' in globals() else dataset

# Числовые признаки
X_num = source_df.drop(columns=[TARGET]).select_dtypes(include=[np.number])
y = source_df[TARGET]

# Импутация оставшихся пропусков медианой (на всякий случай)
imputer = SimpleImputer(strategy='median')
X_num_imp = pd.DataFrame(imputer.fit_transform(X_num), columns=X_num.columns, index=X_num.index)

# 1) Считаем баллы для всех фич
all_selector = SelectKBest(score_func=f_regression, k='all')
all_selector.fit(X_num_imp, y)

scores_df = (
    pd.DataFrame({'feature': X_num_imp.columns, 'score': all_selector.scores_})
      .sort_values('score', ascending=False)
      .reset_index(drop=True)
)
print('Оценки информативности (f_regression), по убыванию:')
print(scores_df)

# Визуализация
plt.figure(figsize=(10, 6))
sns.barplot(data=scores_df, x='score', y='feature', color='#1f77b4')
plt.title('SelectKBest: f_regression scores')
plt.tight_layout()
plt.show()

# 2) Выбор ТОП-K признаков
K = 5
selector = SelectKBest(score_func=f_regression, k=K)
selector.fit(X_num_imp, y)
selected_features = X_num_imp.columns[selector.get_support()].tolist()
print(f'Топ-{K} признаков:')
print(selected_features)
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Удаление нерелевантных признаков, как на слайде
cols_to_drop = ['X2 house age', 'X1 transaction date']

# Работаем с расширенным набором, если он есть; иначе — с исходным
df_ref = dataset_fe if 'dataset_fe' in globals() else dataset

print('До:', df_ref.shape)
df_ref.drop(columns=[c for c in cols_to_drop if c in df_ref.columns], axis=1, inplace=True)
print('После:', df_ref.shape)
print('Текущие столбцы:')
print(df_ref.columns.tolist())

# Если работали с исходным датасетом, синхронизируем переменную
if 'dataset_fe' not in globals():
    dataset = df_ref
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Стандартизация числовых признаков (z = (x - mean) / std)
TARGET = 'Y house price of unit area'

# Базовый датафрейм для трансформации: используем расширенный, если есть
base_df = dataset_fe if 'dataset_fe' in globals() else dataset

# Отбираем числовые признаки, исключая целевую
feature_cols = base_df.select_dtypes(include=[np.number]).columns.tolist()
if TARGET in feature_cols:
    feature_cols.remove(TARGET)

scaler = StandardScaler()
scaled = scaler.fit_transform(base_df[feature_cols])
standardized_df = pd.DataFrame(scaled, columns=feature_cols, index=base_df.index)

print('Стандартизированы признаки:', feature_cols)
print('Форма:', standardized_df.shape)
print(standardized_df.head())

# Быстрая проверка средних и СКО (должны быть ~0 и ~1)
means = standardized_df.mean().round(4)
stdevs = standardized_df.std(ddof=0).round(4)
print('\nСредние по столбцам (ожид. ≈ 0):')
print(means)
print('\nСт. отклонения (ожид. ≈ 1):')
print(stdevs)
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Train/Validation/Test split: 60% / 20% / 20%

TARGET = 'Y house price of unit area'

# Источник данных: используем расширенный набор, если он уже есть
src = dataset_fe if 'dataset_fe' in globals() else dataset

X = src.drop(columns=[TARGET])
y = src[TARGET]

# 1) Test split (20%)
seed = 42
test_size = 0.2
val_size = 0.25  # 25% от train -> итог 60/20/20

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=seed
)

# 2) Validation split из обучающей части
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=seed
)

print('Shapes:')
print('X_train:', X_train.shape, 'X_val:', X_val.shape, 'X_test:', X_test.shape)
print('y_train:', y_train.shape, 'y_val:', y_val.shape, 'y_test:', y_test.shape)

print('\nX_train head:')
print(X_train.head())
print('\nX_val head:')
print(X_val.head())
print('\nX_test head:')
print(X_test.head())

print('\ny_train head:')
print(y_train.head())
print('\ny_val head:')
print(y_val.head())
print('\ny_test head:')
print(y_test.head())
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Сравнение MLP и простой RNN на train/val для исходного и инженерного наборов
np.random.seed(42)
try:
    tf.random.set_seed(42)
except Exception:
    pass

TARGET = 'Y house price of unit area'

# Проверим, что разбиение уже сделано (из предыдущей ячейки)
assert 'X_train' in globals() and 'X_val' in globals(), 'Сначала запустите ячейку с train/val/test split.'
train_idx = X_train.index
val_idx = X_val.index

# Подготовим словарь наборов
datasets_map = {}
datasets_map['original'] = dataset.copy()
if 'dataset_fe' in globals():
    datasets_map['engineered'] = dataset_fe.copy()

results = []

for name, df in datasets_map.items():
    # берем только числовые признаки
    X_all = df.drop(columns=[TARGET]).select_dtypes(include=[np.number])
    y_all = df[TARGET]

    # одинаковые индексы разбиения
    X_tr = X_all.loc[train_idx]
    y_tr = y_all.loc[train_idx]
    X_va = X_all.loc[val_idx]
    y_va = y_all.loc[val_idx]

    # Импутация по train и стандартизация по train
    imputer = SimpleImputer(strategy='median')
    scaler = StandardScaler()

    X_tr_imp = imputer.fit_transform(X_tr)
    X_va_imp = imputer.transform(X_va)

    X_tr_std = scaler.fit_transform(X_tr_imp)
    X_va_std = scaler.transform(X_va_imp)

    input_dim = X_tr_std.shape[1]

    # 1) Полносвязная регрессионная модель (MLP)
    mlp = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])
    mlp.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')
    es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    mlp.fit(X_tr_std, y_tr.values, validation_data=(X_va_std, y_va.values),
            epochs=300, batch_size=32, verbose=0, callbacks=[es])

    y_tr_pred = mlp.predict(X_tr_std, verbose=0).ravel()
    y_va_pred = mlp.predict(X_va_std, verbose=0).ravel()
    rmse_tr = float(np.sqrt(mean_squared_error(y_tr, y_tr_pred)))
    r2_tr = float(r2_score(y_tr, y_tr_pred))
    rmse_va = float(np.sqrt(mean_squared_error(y_va, y_va_pred)))
    r2_va = float(r2_score(y_va, y_va_pred))
    results.append({'dataset': name, 'model': 'MLP', 'rmse_train': rmse_tr, 'r2_train': r2_tr, 'rmse_val': rmse_va, 'r2_val': r2_va})

    # 2) Простая рекуррентная сеть для регрессии (SimpleRNN)
    # Представим признаки как одношаговую последовательность длиной input_dim
    X_tr_seq = X_tr_std.reshape((-1, input_dim, 1))
    X_va_seq = X_va_std.reshape((-1, input_dim, 1))

    rnn = keras.Sequential([
        layers.Input(shape=(input_dim, 1)),
        layers.SimpleRNN(32, activation='tanh'),
        layers.Dense(1)
    ])
    rnn.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')
    es2 = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    rnn.fit(X_tr_seq, y_tr.values, validation_data=(X_va_seq, y_va.values),
            epochs=300, batch_size=32, verbose=0, callbacks=[es2])

    y_tr_pred = rnn.predict(X_tr_seq, verbose=0).ravel()
    y_va_pred = rnn.predict(X_va_seq, verbose=0).ravel()
    rmse_tr = float(np.sqrt(mean_squared_error(y_tr, y_tr_pred)))
    r2_tr = float(r2_score(y_tr, y_tr_pred))
    rmse_va = float(np.sqrt(mean_squared_error(y_va, y_va_pred)))
    r2_va = float(r2_score(y_va, y_va_pred))
    results.append({'dataset': name, 'model': 'SimpleRNN', 'rmse_train': rmse_tr, 'r2_train': r2_tr, 'rmse_val': rmse_va, 'r2_val': r2_va})

# Сводная таблица
res_df = pd.DataFrame(results)
print(res_df.sort_values(['dataset', 'model']).to_string(index=False))
# ### END YOUR CODE
